# Assignment 2

As before, if a question can be answered with 'yes/no', or a numeric value, you may simply state as much. If you incorporate code from the internet (which is not required and generally not advisable), please cite the source within your code (providing a URL is sufficient).

We will go through comparable code and concepts in the live learning sessions. If you run into trouble, start by using the help `help()` function in Python, to get information about the datasets and function in question. The internet is also a great resource when coding (though note that no outside searches are required by the assignment!). If you do incorporate code from the internet, please cite the source within your code (providing a URL is sufficient).

Please bring questions that you cannot work out on your own to office hours, work periods or share with your peers on Slack. We will work with you through the issue.

If you like, you may collaborate with others in the cohort. If you choose to do so, please indicate with whom you have worked with in your pull request by tagging their GitHub username. Separate submissions are required.

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Import specific objects
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from ISLP import load_data
from ISLP import confusion_table

### Question 1: Classification using KNN

We'll now use the `Caravan` dataset from the `ISLP` package. (You may use `Caravan.describe()` to review details of the dataset.) In this dataset, the response variable of interest is `Purchase`, which indicates if a given customer purchased a caravan insurance policy. We will simultaneously use all other variables in the dataset to predict the response variable.

In [4]:
# Load the "Caravan" dataset using the "load_data" function from the ISLP package
Caravan = load_data('Caravan')

# Add your code here
count = (Caravan['Purchase'] == 'Yes').sum()
print ( count )
print ( Caravan.head(50) )
Caravan.describe().columns

348
    MOSTYPE  MAANTHUI  MGEMOMV  MGEMLEEF  MOSHOOFD  MGODRK  MGODPR  MGODOV  \
0        33         1        3         2         8       0       5       1   
1        37         1        2         2         8       1       4       1   
2        37         1        2         2         8       0       4       2   
3         9         1        3         3         3       2       3       2   
4        40         1        4         2        10       1       4       1   
5        23         1        2         1         5       0       5       0   
6        39         2        3         2         9       2       2       0   
7        33         1        2         3         8       0       7       0   
8        33         1        2         4         8       0       1       3   
9        11         2        3         3         3       3       5       0   
10       10         1        4         3         3       1       4       1   
11        9         1        3         3         3       1  

Index(['MOSTYPE', 'MAANTHUI', 'MGEMOMV', 'MGEMLEEF', 'MOSHOOFD', 'MGODRK',
       'MGODPR', 'MGODOV', 'MGODGE', 'MRELGE', 'MRELSA', 'MRELOV', 'MFALLEEN',
       'MFGEKIND', 'MFWEKIND', 'MOPLHOOG', 'MOPLMIDD', 'MOPLLAAG', 'MBERHOOG',
       'MBERZELF', 'MBERBOER', 'MBERMIDD', 'MBERARBG', 'MBERARBO', 'MSKA',
       'MSKB1', 'MSKB2', 'MSKC', 'MSKD', 'MHHUUR', 'MHKOOP', 'MAUT1', 'MAUT2',
       'MAUT0', 'MZFONDS', 'MZPART', 'MINKM30', 'MINK3045', 'MINK4575',
       'MINK7512', 'MINK123M', 'MINKGEM', 'MKOOPKLA', 'PWAPART', 'PWABEDR',
       'PWALAND', 'PPERSAUT', 'PBESAUT', 'PMOTSCO', 'PVRAAUT', 'PAANHANG',
       'PTRACTOR', 'PWERKT', 'PBROM', 'PLEVEN', 'PPERSONG', 'PGEZONG',
       'PWAOREG', 'PBRAND', 'PZEILPL', 'PPLEZIER', 'PFIETS', 'PINBOED',
       'PBYSTAND', 'AWAPART', 'AWABEDR', 'AWALAND', 'APERSAUT', 'ABESAUT',
       'AMOTSCO', 'AVRAAUT', 'AAANHANG', 'ATRACTOR', 'AWERKT', 'ABROM',
       'ALEVEN', 'APERSONG', 'AGEZONG', 'AWAOREG', 'ABRAND', 'AZEILPL',
       'APLEZIER', 'AFIETS',

Before fitting any model, it is essential to understand our data. Answer the following questions about the `Caravan` dataset (Hint: use `print` and `describe`):  
_(i)_ How many observations (rows) does the dataset contain?    


In [13]:
print( "Number of raws is ", Caravan.shape[0] )          

Number of raws is  5822


_(ii)_ How many variables (columns) does the dataset contain?    


In [12]:
print( "Number of columns is ", Caravan.shape[1] )

Number of columns is  86


_(iii)_ What 'variable' type is the response variable `Purchase` (e.g., 'character', 'factor', 'numeric', etc)? What are the 'levels' of the variable?    

Response `Purchase` is bolean, qualitative variable type.
There is two levels of variable :   0 - "No"
                                    1 - "Yes"

_(iv)_ How many predictor variables do we have (Hint: all variables other than `Purchase`)?  

In [14]:
# Add your code here
print( "Number of predictors is ", Caravan.shape[1] - 1 )             

Number of predictors is  85


Next, we must preform 'pre-processing' or 'data munging', to prepare our data for classification/prediction. For KNN, there are three essential steps. A first essential step is to 'standardize' the predictor variables. We can achieve this using the `scaler` method, provided as follows:

In [17]:
# Select predictors (excluding the 86th column)
predictors = Caravan.iloc[:, :-1]

# Standardize the predictors
scaler = StandardScaler()
predictors_standardized = pd.DataFrame(scaler.fit_transform(predictors), columns=predictors.columns)

# Display the head of the standardized predictors
print(predictors_standardized.head(50))

     MOSTYPE  MAANTHUI   MGEMOMV  MGEMLEEF  MOSHOOFD    MGODRK    MGODPR  \
0   0.680906 -0.272580  0.406697 -1.216964  0.779405 -0.694311  0.217444   
1   0.992297 -0.272580 -0.859500 -1.216964  0.779405  0.302552 -0.365410   
2   0.992297 -0.272580 -0.859500 -1.216964  0.779405 -0.694311 -0.365410   
3  -1.187437 -0.272580  0.406697  0.010755 -0.970980  1.299414 -0.948264   
4   1.225840 -0.272580  1.672893 -1.216964  1.479559  0.302552 -0.365410   
5  -0.097570 -0.272580 -0.859500 -2.444683 -0.270826 -0.694311  0.217444   
6   1.147992  2.191644  0.406697 -1.216964  1.129482  1.299414 -1.531118   
7   0.680906 -0.272580 -0.859500  0.010755  0.779405 -0.694311  1.383152   
8   0.680906 -0.272580 -0.859500  1.238473  0.779405 -0.694311 -2.113972   
9  -1.031742  2.191644  0.406697  0.010755 -0.970980  2.296276  0.217444   
10 -1.109590 -0.272580  1.672893  0.010755 -0.970980  0.302552 -0.365410   
11 -1.187437 -0.272580  0.406697  0.010755 -0.970980  0.302552 -0.948264   
12  0.680906

_(v)_ Why is it important to standardize the predictor variables?  
Each predictor has its own scale. While one predictor could have a very large number, others could be very small. The difference in scaling can lead to errors and increase them. So, the goal is to normalize every predictor to the same scale between 0 and 1.

_(vi)_ Why did we elect not to standard our response variable `Purchase`? </br>
Our response variable 'Purchase' can only have values of 0 or 1. So, it can be considered as already normalized.
Normalization of response could complicate the interpetation.


_(vii)_ A second essential step is to set a random seed. Do so below (Hint: use the `random.seed` function). Why is setting a seed important? Is the particular seed value important? Why or why not?</br>
Using the function random.seed ensures that we will obtain the same results from random operations when we use it with the same particular value. It's important to use the same value, not just any number. This feature is crucial for comparing, testing, and debugging results.

In [52]:
# Add your code here
np.random.seed(20)

_(viii)_ A third essential step is to split our standardized data into separate training and testing sets. We will split into 75% training and 25% testing. The provided code randomly partitions our data, and creates linked training sets for the predictors and response variables. Extend the code to create a non-overlapping test set for the predictors and response variables.

In [48]:
# Create a random vector of True and False values
split = np.random.choice([True, False], size=len(predictors_standardized), replace=True, p=[0.75, 0.25])

# Define the training set for X (predictors)
training_X = predictors_standardized[split]

# Define the training set for Y (response)
training_Y = Caravan.loc[split, 'Purchase']

# Define the testing set for X (predictors)
testing_X = predictors_standardized[~split]

# Define the testing set for Y (response)
testing_Y = Caravan.loc[~split, 'Purchase']


_(ix)_ We are finally set to fit the KNN model. In Python, we can use the `KNeighborsClassifier()` function. Fit the KNN with k=1. (You may review arguments to knn by typing `help(knn.fit)`). 

In [49]:
# Add your code here
knn = KNeighborsClassifier( n_neighbors=1 )
knn.fit( training_X, training_Y )
knn_pred = knn.predict( testing_X )
confusion_table( knn_pred, testing_Y )

Truth,No,Yes
Predicted,,
No,1329,67
Yes,96,14


Using your fit model, answer the following questions:   
_(x)_ What is the prediction accuracy? (Hint: use the `score` method, and compare your model to `testing_Y`)</br> Accuracy rate is almost 89% which is very good.


_(xi)_ What is the predictor error ? (Hint: compute it from the accuracy)

In [50]:
# prediction accuracy rate
acc_np = np.mean(knn_pred == testing_Y)
acc_score = knn.score(testing_X, testing_Y)
acc_np, acc_score

(0.8917662682602921, 0.8917662682602921)

In [51]:
# prediction error rate
pred_err_rate = 1 - acc_score
pred_err_rate

0.10823373173970785

_(xii)_ How does this prediction error/accuracy compare to what could be achieved via random guesses? To answer this, consider the percent of customers in the `Caravan` dataset who actually purchase insurance, computed below:

In [53]:
# Calculate the percentage of customers who purchase insurance
percentage_purchase = (Caravan['Purchase'].eq('Yes').sum() / len(Caravan['Purchase'])) * 100

print(percentage_purchase)

5.977327378907591


Our dataset contains only 6% of "YES" cases, so guessing a "Yes" case correctly is very difficult. </br>
Our model has an accuracy of 17% ( 14 / ( 67+14 ) * 100 )for predicting "Yes" cases.

If we consider guessing "YES/NO" as a 50% probability for each. 6% "Yes" cases. Than general accuracy for both (YES\No) cases would be 56%  compared to the model's prediction accuracy of 89%.

The model's accuracy is significantly higher, even before finding the best value of K. Therefore, our model is much more efficient.

_(xiii)_ Fit a second KNN model, with $K=3$. Does this model perform better (i.e., have higher accuracy, compared to a random guess)?

In [67]:
# Your code here
knn3 = KNeighborsClassifier( n_neighbors=3 )
knn3.fit( training_X, training_Y )
knn_pred = knn3.predict( testing_X )

acc_np = np.mean(knn_pred == testing_Y)
print ( acc_np )
confusion_table( knn_pred, testing_Y )

0.9329349269588313


Truth,No,Yes
Predicted,,
No,1400,76
Yes,25,5


 $K=3$ increases accuracy by 4%, which is better than the same model with $K=1$ and significantly greater than random guessing.

# Criteria

|Criteria            |Complete           |Incomplete          |
|--------------------|---------------|--------------|
|Classification using KNN|All steps are done correctly and the answers are correct.|At least one step is done incorrectly leading to a wrong answer.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/applied_statistical_concepts/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
